In [1]:
import mysql.connector
from decimal import Decimal
from datetime import datetime

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ALKATRAZE17omega",
    database="mydb"
)

mycursor = mydb.cursor(buffered=True)

In [2]:

def pizza_ingredient(pizza):
    querry = """
    SELECT 
    ingredient_pizza.idIngredient, ingredient.vegetarian
    FROM 
    ingredient_pizza JOIN ingredient 
    ON ingredient_pizza.idIngredient = ingredient.idIngredient 
    WHERE idPizza = '{}'
    """.format(pizza)
    mycursor.execute(querry)
    
    my_result = mycursor.fetchall()
    if len(my_result) == 0: #if there are no igredients then the pizza was not found in the ingredient list
        return None
    
    ingredients, veg = zip(*my_result)
    
    #convert from tuple to list
    ingredients = list(ingredients)
    veg = list(veg)
    
    #append vegetarian boolean at the end of the ingredient list
    for i, x in enumerate(veg):
        if x == 0:
            ingredients.append(0)
            break
        elif i == len(veg) - 1:
            ingredients.append(1)
            
    return ingredients

In [3]:
#print(pizza_ingredient("forbigboys"))

In [4]:
#returns a list of (name, price) tuples for item_type 'pizza', 'drink' and 'dessert'
def get_menu_listing(item_type):
    
    #func to add vat and margin
    def mark_up(tuple_list):
        mark_up_list = []
        for name, price in tuple_list:
            price = price * Decimal('1.09')#prices in the database are stored as decimal rather than float
            price = price * Decimal('1.4')
            mark_up_list.append((name, round(price, 2)))

        return mark_up_list
    
    # gets menu items from drink or deserts with their price.
    def get_direct_list(item_type):
        primary_key = "id" + item_type.capitalize()#our naming convention for primary keys

        querry = " SELECT {}, price FROM {}".format(primary_key, item_type)
        mycursor.execute(querry)

        my_result = mycursor.fetchall()
        return mark_up(my_result)
    
    #get's the price of pizza through ingredient prices
    def get_pizza_list():
        querry = """
        SELECT idPizza, SUM(price) FROM 
        (SELECT 
        ingredient_pizza.idPizza, 
        ingredient.price
        FROM 
        ingredient_pizza INNER JOIN ingredient
        ON
        ingredient_pizza.idIngredient = ingredient.idIngredient) AS pizza_ingredient_price
        GROUP BY idPizza
        """

        mycursor.execute(querry)

        my_result = mycursor.fetchall()

        return mark_up(my_result)
    
    #decide whether to us the pizza lister or other lister
    primary_key = "id" + item_type.capitalize()#our naming convention for primary keys
    if item_type == 'pizza':
        return get_pizza_list()
    else:
        return get_direct_list(item_type)

In [5]:
def check_coupon(coupon):
    querry = "SELECT code FROM discount WHERE code = {}".format(code)
    mycursor.execute(querry)
    my_result = mycursor.fetchall()

    if len(my_result) == 0:
        return False
    else:
        return True

#calculate order total
def order_total(order_list, valid_coupon=False):

    total = 0
    for name, price, quantity in order_list:
        total += price * quantity
        
    if valid_coupon:
        total = total * 0.9

    return total

def excute_order(order_list, idCustomer, use_coupon=None, reward_coupon=None):
    #get next idOrder auto-increment
    id_querry = """
    SELECT `AUTO_INCREMENT`
    FROM  INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'mydb'
    AND   TABLE_NAME   = 'orders';"""

    mycursor.execute(id_querry)
    my_result = mycursor.fetchone()
    idOrder = my_result[0]

    #insert in order table
    #check coupon values
    if use_coupon is None:
        use_coupon = 'NULL'
    if reward_coupon is None:
        reward_coupon = 'NULL'

    #insert the order
    order_querry = """
    INSERT INTO 
    orders (order_date_time, use_coupon, reward_coupon, idCustomer)
    VALUES (NOW(), {}, {}, {})
    """.format(use_coupon,reward_coupon,idCustomer)

    mycursor.execute(order_querry)

    #add items to order
    for item, quantity in order_list:

        item_querry = """
        INSRT INTO
        menu_order (idMenu, idOrder, quantity)
        VALUES ('{}','{}','{}')
        """.format(item, idOrder, quantity)

        mycursor.execute(item_querry)
    #commit the order and all items in one transaction
    mydb.commit()

In [6]:
def get_pizza_count(idCustomer):
    pass

In [10]:
def get_next_id():
    id_querry = """
    SELECT `AUTO_INCREMENT`
    FROM  INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'mydb'
    AND   TABLE_NAME   = 'customer';""" 
    
    mycursor.execute(id_querry)
    my_result = mycursor.fetchone()
    idOrder = my_result[0]
    print(idOrder)

In [31]:
def get_order_list(idCustomer):
    querry = """
    SELECT idOrder, order_date_time 
    FROM orders 
    WHERE idCustomer = {}
    """.format(idCustomer)
    
    mycursor.execute(querry)
    my_result = mycursor.fetchall()
    
    id_order, time = zip(*my_result)
    id_order = list(id_order)
    time = list(time)
    str_time = []
    for t in time:
        str_time.append(t.strftime("%m/%d/%Y, %H:%M"))
    
    final_result = list(zip(id_order, str_time))
    
    return final_result

In [33]:
def get_order_total(idOrder):
    item_querry = """
    """

[(3, '10/13/2022, 11:27'),
 (6, '10/13/2022, 11:32'),
 (7, '10/13/2022, 11:34'),
 (8, '10/13/2022, 11:35'),
 (9, '10/13/2022, 11:35'),
 (10, '10/13/2022, 11:35'),
 (11, '10/13/2022, 11:35'),
 (12, '10/13/2022, 11:36'),
 (13, '10/13/2022, 11:36'),
 (14, '10/13/2022, 11:40')]